# **Reproducing Kernel Hilbert Space**

## Import packages

In [2]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from IPython.display import Markdown
from IPython.display import display, Markdown

## Load functions

Please follows the following steps:
1. Initiate by executing the notebook titled 'Kernel_function_def.ipynb'. \\
2. Upon completion, the notebook will automatically save as 'kernel_functions.py'. \\
3. Load the kernel evaluation functions into the current notebook by running the command %run kernel_functions.py as shown in code snippet [1]. \\


In [3]:
# Load the python file containing the kernel functions
%run kernel_functions.py

## Data

Consider that you have training inputs/samples $(x_i)_{i=1}^n \in \mathbb{R}^d$ and targets $(y_i)_{i=1}^n \in \mathbb{R}$, where $d$ is the number of features. We can define the *design matrix* and the *output vector* respectively as

\begin{equation}
\textbf{X} := \begin{bmatrix} x_1^\top \\ \vdots \\ x_n^\top\end{bmatrix} \in \mathbb{R}^{n \times d}, \qquad \textbf{y} := \begin{bmatrix} y_1 \\ \vdots \\ y_n\end{bmatrix} \in \mathbb{R}^{n}.
\end{equation}

Now, suppose that we have $(z_i)_{i=1}^m \in \mathbb{R}^d$ which are the *test inputs* and we want to predict our models on this set of test data. We define the following

\begin{equation}
\textbf{Z} := \begin{bmatrix} z_1^\top \\ \vdots \\ z_m^\top\end{bmatrix} \in \mathbb{R}^{m \times d}, \qquad {\textbf{y}}^\text{pred} := \begin{bmatrix} {f}^\star(z_1) \\ \vdots \\ f^\star(z_m) \end{bmatrix} \in \mathbb{R}^{m},
\end{equation}

where ${\textbf{y}}^\text{pred}$ is the vector of *predicted outputs*. Given the actual targets for the test data set, which we denote as

\begin{equation}
\textbf{y}^{\text{true}} := \begin{bmatrix} y_1^\text{true} \\ \vdots \\ y_m^\text{true} \end{bmatrix} \in \mathbb{R}^{m},
\end{equation}

we can denote the vector of *residuals* as

\begin{equation}
\textbf{ɛ} := {\textbf{y}}^\text{pred} - \textbf{y}^\text{true}.
\end{equation}



## Data preparation

In [4]:
# Read the data file
df = pd.read_csv('data.csv')

# Extract columns containing the feature vectors
cols = [1,2,3,4,5,6]

#  Store the input feature vectors in X
X = df[df.columns[cols]]

# Store the target variables in y
y = df[df.columns[[7]]]

# Convert to numpy array
X = X.to_numpy()
y = y.to_numpy()

# Print the first few rows of the DataFrame
print("DataFrame sample:")
print(df.head(3))

DataFrame sample:
   No  X1 transaction date  X2 house age  \
0   1             2012.917          32.0   
1   2             2012.917          19.5   
2   3             2013.583          13.3   

   X3 distance to the nearest MRT station  X4 number of convenience stores  \
0                                84.87882                               10   
1                               306.59470                                9   
2                               561.98450                                5   

   X5 latitude  X6 longitude  Y house price of unit area  
0     24.98298     121.54024                        37.9  
1     24.98034     121.53951                        42.2  
2     24.98746     121.54391                        47.3  


We now split the dataset into a training set and a testing set. We use the following variables:

1. X_train : matrix $\textbf{X}$

2. X_test : matrix $\textbf{Z}$

3. y_train : vector $\textbf{y}$

4. y_test : vector $\textbf{y}^\text{true}$

Before using the models, you should first standardize the training dataset and the test dataset.

In [5]:
np.random.seed(1)

# Split X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

# Calculate mean and standard deviation of each column
mean = np.mean(X_train, axis = 0)
standard_deviation = np.std(X_train, axis = 0)

# Standardize X_train and X_test
X_train = (X_train - mean)/standard_deviation
X_test = (X_test - mean)/standard_deviation

## Error metrics for model performance comparison

For comparison of model performance, we consider six types of errors:


 1. $\ell^1$ norm of $\textbf{ɛ}$: $\sum_{i=1}^m |ɛ_i|$.


 2. Mean absolute error of $\textbf{ɛ}$: $\frac{1}{m}\sum_{i=1}^m |ɛ _i|$.



 3. Relative absolute error of $\textbf{ɛ}$ : $\dfrac{\sum_{i=1}^m |ɛ _i|}{\sum_{i=1}^m |y_i^\text{true} - \overline{y^\text{true}}|}$.



 4. $\ell^2$ norm of $\textbf{ɛ}$: $\left(\sum_{i=1}^m |ɛ_i|^2\right)^{1/2}$.


 5. Mean squared error of $\textbf{ɛ}$: $\frac{1}{m}\sum_{i=1}^m |ɛ_i|^2$.



 6. Relative squared error of $\textbf{ɛ}$: $\dfrac{\sum_{i=1}^m |ɛ _i|^2}{\sum_{i=1}^m |y_i^\text{true} - \overline{y^\text{true}}|^2}$.


In the above formulae, $\overline{y^\text{true}} := \frac{1}{m} \sum_{i=1}^m y_i^\text{true}$.

In [6]:
def errors(y_test, y_pred):
    """
    Calculate various error metrics for model performance comparison.

    Parameters:
    y_test (iterable): True values.
    y_pred (iterable): Predicted values.

    Returns:
    dict: A dictionary containing the calculated error metrics.
    """
    # Calculate the l_1 norm
    error_1 = np.sum(np.abs(y_pred - y_test))
    print(np.mean(np.abs(y_pred-y_test)))

    # Calculate the mean absolute error
    mean_absolute_error = np.mean(np.abs(y_pred - y_test))

    # Calculate the relative absolute 
    y_true_mean = np.mean(y_test) # Calculate the mean of y_true
    relative_absolute_error = np.sum(np.abs(y_pred - y_test)) / np.sum(np.abs(y_test - y_true_mean))
        
    # Calculate the l_2 norm
    error_2 = np.sqrt(np.sum((y_pred - y_test)** 2))

    # Calculate the mean squared error
    mean_squared_error = np.mean((y_pred - y_test) ** 2)

    # Calculate the relative squared error
    relative_squared_error = np.sum(np.abs(y_pred - y_test) ** 2) / np.sum((y_test - y_true_mean) ** 2)
    
    return (error_1, mean_absolute_error, relative_absolute_error, error_2, mean_squared_error, relative_squared_error)

## Linear regression

The optimal parameter $\mathbf{\beta}^\star$ obtained from

\begin{equation}
  \beta^\star := \underset{\beta \in \mathbb{R}^n}{\operatorname{argmin}} \frac{1}{n} \left\|\textbf{y} - \textbf{X}\beta\right\|_2^2
\end{equation}

is given by

\begin{equation}
\beta^\star = (\textbf{X}^\top \textbf{X})^{-1} \textbf{X}^\top \textbf{y}.
\end{equation}

We predict at the test inputs $\textbf{Z}$ by

\begin{equation}
\textbf{y}^\text{pred} = \textbf{Z}\beta^\star.
\end{equation}

In [7]:
def linear_regression():
    X_train_T = X_train.T
    ones_column = np.ones((X_train.shape[0], 1))  
    X_train_with_intercept = np.hstack((ones_column, X_train))  
    beta_star = np.linalg.inv(X_train_with_intercept.T @ X_train_with_intercept) @ X_train_with_intercept.T @ y_train

    # Predict on the test set
    ones_column_test = np.ones((X_test.shape[0], 1))  
    X_test_with_intercept = np.hstack((ones_column_test, X_test))  
    y_pred = X_test_with_intercept @ beta_star

    # print(y_pred)
    return y_pred

In [8]:
# Predict y using linear regression
y_pred = linear_regression()

# Obtain error metrics
error_1, mean_absolute_error, relative_absolute_error, \
error_2, mean_squared_error, relative_squared_error = errors(y_test, y_pred)

# Display the error metrics
stats = np.array([error_1, mean_absolute_error, relative_absolute_error, error_2, mean_squared_error, relative_squared_error])
stats = pd.DataFrame(stats, columns = ['Statistics for linear regression'])
stats.index = [r"$\ell^1$ Error", 'Mean Absolute Error', 'Relative Absolute Error', \
               r"$\ell^2$ Error", 'Mean Squared Error', 'Relative Squared Error']
display(Markdown(stats.to_markdown()))

6.274984907782409


|                         |   Statistics for linear regression |
|:------------------------|-----------------------------------:|
| $\ell^1$ Error          |                         784.373    |
| Mean Absolute Error     |                           6.27498  |
| Relative Absolute Error |                           0.543918 |
| $\ell^2$ Error          |                         114.873    |
| Mean Squared Error      |                         105.566    |
| Relative Squared Error  |                           0.458135 |

## Ridge regression

The optimal parameter $\mathbf{\beta}^\star$ obtained from

\begin{equation}
  \beta^\star := \underset{\beta \in \mathbb{R}^n}{\operatorname{argmin}} \frac{1}{n} \left\|\textbf{y} - \textbf{X}\beta\right\|_2^2 + \mu \|\beta\|_2^2
\end{equation}

is given by

\begin{equation}
\beta^\star = \left(\textbf{X}^\top \textbf{X} + n \mu \textbf{I}\right)^{-1} \textbf{X}^\top \textbf{y}.
\end{equation}

We predict at the test inputs $\textbf{Z}$ by

\begin{equation}
\textbf{y}^\text{pred} = \textbf{Z}\beta^\star.
\end{equation}

In [9]:
def ridge_regression_with_intercept(mu):
    X_train_intercept = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
    X_test_intercept = np.hstack([np.ones((X_test.shape[0], 1)), X_test])
    
    n = X_train_intercept.shape[0]
    X_train_T = X_train_intercept.T
    I = np.identity(X_train_intercept.shape[1])
    I[0, 0] = 0 
    
    beta_star = np.linalg.inv(X_train_T @ X_train_intercept + n * mu * I) @ X_train_T @ y_train
    y_pred = X_test_intercept @ beta_star

    return y_pred

In [10]:
# Choose a penalty factor below. You can try with mu = 0.01, 0.1, 1, 10, 100
mu = 0.1
# Predict y using ridge regression
y_pred = ridge_regression_with_intercept(mu)

# Obtain error metrics
error_1, mean_absolute_error, relative_absolute_error, \
error_2, mean_squared_error, relative_squared_error = errors(y_test, y_pred)

# Display the error metrics
stats = np.array([error_1, mean_absolute_error, relative_absolute_error, error_2, mean_squared_error, relative_squared_error])
stats = pd.DataFrame(stats, columns=['Statistics for ridge regression'])
stats.index = [r"$\ell^1$ Error", 'Mean Absolute Error', 'Relative Absolute Error', r"$\ell^2$ Error", 'Mean Squared Error', 'Relative Squared Error']
display(Markdown(stats.to_markdown()))

6.360906919486817


|                         |   Statistics for ridge regression |
|:------------------------|----------------------------------:|
| $\ell^1$ Error          |                        795.113    |
| Mean Absolute Error     |                          6.36091  |
| Relative Absolute Error |                          0.551366 |
| $\ell^2$ Error          |                        116.041    |
| Mean Squared Error      |                        107.723    |
| Relative Squared Error  |                          0.467499 |

## Kernel ridge regression

Using the *Representer theorem*, the optimal solution to the learning problem

\begin{equation}
f^\star(x) := \underset{f \in \mathcal{H}}{\operatorname{argmin}} \frac{1}{n} \sum_{i=1}^n \Big(y_i - f(x_i)\Big)^2 + \mu \|f\|_\mathcal{H}^2
\end{equation}

is given by

\begin{equation}
f^\star(x) = \sum_{i=1}^n \alpha_i^\star K(x_i, x),
\end{equation}

The optimal coefficients $\alpha_i^\star, \, 1\leq i \leq n$ are obtained as

\begin{equation}
 \alpha^\star := \begin{bmatrix} \alpha_1^\star \\ \vdots \\ \alpha_n^\star \end{bmatrix} = \left(\textbf{K} + n\mu \textbf{I}\right)^{-1} \textbf{y} \in \mathbb{R}^n,
\end{equation}

where

\begin{equation}
\textbf{K} :=
\begin{bmatrix}
K(x_1, x_1) & \cdots & K(x_1, x_n)\\
\vdots & \ddots & \vdots \\
K(x_n, x_1) & \cdots & K(x_n,x_n)
\end{bmatrix} \in \mathbb{R}^{n \times n}
\end{equation}


is the *kernel matrix*.

We predict at the test inputs $\textbf{Z}$ by

\begin{equation}
\textbf{y}^\text{pred} = \textbf{G} \alpha^\star,
\end{equation}

where

\begin{equation}
\textbf{G} :=
\begin{bmatrix}
K(z_1, x_1) & \cdots & K(z_1, x_n) \\
\vdots & \ddots & \vdots \\
K(z_m, x_1) & \cdots & K(z_m, x_n)
\end{bmatrix} \in \mathbb{R}^{m \times n}.
\end{equation}

In [11]:
def kernel_ridge_regression(X_train, y_train, X_test, kernel_type='convex', mu=0.1, weights=None, length_scales=None, kernel_choice_type=None, kernel_length_scale=1.0):
    
    n = X_train.shape[0]

    if kernel_type == 'convex':
        if weights is None or length_scales is None:
            raise ValueError("Weights and length_scales must be provided for convex kernel.")
        K, G = convex_kernel(X_train, X_test, weights, length_scales)
    elif kernel_type == 'product':
        if length_scales is None:
            raise ValueError("Length_scales must be provided for product kernel.")
        K, G = product_kernel(X_train, X_test, length_scales)
    elif kernel_type == 'choice':
        if kernel_choice_type is None:
            raise ValueError("kernel_choice_type must be provided for kernel_choice.")
        K, G = kernel_choice(X_train, X_test ,kernel_choice_type, kernel_length_scale)
        
    else:
        raise ValueError("Invalid kernel type. Choose 'convex', 'product', or 'choice'.")

    # Compute the optimal coefficients alpha_star
    I = np.identity(n)
    alpha_star = np.linalg.inv(K + n * mu * I) @ y_train

    # Predict the values for the test set
    y_pred = G @ alpha_star
    return y_pred

In [12]:
# Choose a penalty factor below. You can try with mu = 0.01, 0.1, 1, 10, 100
mu =0.1

# Predict y using kernel ridge regression (single kernel)   
y_pred = kernel_ridge_regression(X_train, y_train, X_test, mu=mu, kernel_type='choice', kernel_choice_type="RBF")

# Obtain error metrics
error_1, mean_absolute_error, relative_absolute_error, \
error_2, mean_squared_error, relative_squared_error = errors(y_test, y_pred)

# Display the error metrics
stats = np.array([error_1, mean_absolute_error, relative_absolute_error, error_2, mean_squared_error, relative_squared_error])
stats = pd.DataFrame(stats, columns = ['Statistics for kernel ridge regression'])
stats.index = [r"$\ell^1$ Error", 'Mean Absolute Error', 'Relative Absolute Error', \
               r"$\ell^2$ Error", 'Mean Squared Error', 'Relative Squared Error']
display(Markdown(stats.to_markdown()))

22.672864395408705


|                         |   Statistics for kernel ridge regression |
|:------------------------|-----------------------------------------:|
| $\ell^1$ Error          |                               2834.11    |
| Mean Absolute Error     |                                 22.6729  |
| Relative Absolute Error |                                  1.96529 |
| $\ell^2$ Error          |                                288.126   |
| Mean Squared Error      |                                664.135   |
| Relative Squared Error  |                                  2.88222 |

## KRR with different combination of kernels

We now seek to construct more general kernel functions. For this purpose, refer to the file 'kernel_functions.py' and follow the instructions written there.  Finally, having implemented the kernel function, we want to perform kernel ridge regression.

## KRR with convex combination of kernels

In [13]:
# Choose a penalty factor below. You can try with mu = 0.01, 0.1, 1, 10, 100
mu = 0.1
weights = {'RBF': 0.5, 'Matern': 0.3, 'Laplacian': 0.2}
length_scales = {'RBF': 1.0, 'Matern': 1.0, 'Laplacian': 1.0}

# Predict y using kernel ridge regression (convex combinations)
y_pred = kernel_ridge_regression(X_train, y_train, X_test, kernel_type='convex', mu=mu, weights=weights, length_scales=length_scales)

# Obtain error metrics
error_1, mean_absolute_error, relative_absolute_error, \
error_2, mean_squared_error, relative_squared_error = errors(y_test, y_pred)

# Display the error metrics
stats = np.array([error_1, mean_absolute_error, relative_absolute_error, error_2, mean_squared_error, relative_squared_error])
stats = pd.DataFrame(stats, columns = ['Statistics for kernel ridge regression'])
stats.index = [r"$\ell^1$ Error", 'Mean Absolute Error', 'Relative Absolute Error', \
               r"$\ell^2$ Error", 'Mean Squared Error', 'Relative Squared Error']
display(Markdown(stats.to_markdown()))

22.34963934599475


|                         |   Statistics for kernel ridge regression |
|:------------------------|-----------------------------------------:|
| $\ell^1$ Error          |                               2793.7     |
| Mean Absolute Error     |                                 22.3496  |
| Relative Absolute Error |                                  1.93727 |
| $\ell^2$ Error          |                                284.814   |
| Mean Squared Error      |                                648.951   |
| Relative Squared Error  |                                  2.81632 |

## KRR with product of kernels

In [14]:
# Choose a penalty factor below. You can try with mu = 0.01, 0.1, 1, 10, 100
mu = 0.1
length_scales = {'RBF': 1.0, 'Matern': 1.0, 'Laplacian': 1.0}

# Predict y using kernel ridge regression (convex combinations)
y_pred = kernel_ridge_regression(X_train, y_train, X_test, kernel_type='product', mu=mu, length_scales=length_scales)

# Obtain error metrics
error_1, mean_absolute_error, relative_absolute_error, \
error_2, mean_squared_error, relative_squared_error = errors(y_test, y_pred)

# Display the error metrics
stats = np.array([error_1, mean_absolute_error, relative_absolute_error, error_2, mean_squared_error, relative_squared_error])
stats = pd.DataFrame(stats, columns = ['Statistics for kernel ridge regression'])
stats.index = [r"$\ell^1$ Error", 'Mean Absolute Error', 'Relative Absolute Error', \
               r"$\ell^2$ Error", 'Mean Squared Error', 'Relative Squared Error']
display(Markdown(stats.to_markdown()))

34.785819975601974


|                         |   Statistics for kernel ridge regression |
|:------------------------|-----------------------------------------:|
| $\ell^1$ Error          |                               4348.23    |
| Mean Absolute Error     |                                 34.7858  |
| Relative Absolute Error |                                  3.01525 |
| $\ell^2$ Error          |                                419.599   |
| Mean Squared Error      |                               1408.51    |
| Relative Squared Error  |                                  6.11264 |